# Методы сбора и обработки данных из сети Интернет
### GeekBrains
## Практическое задание урока 5
## Казанцев Виталий

# Вариант I

Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (
*    от кого, 
*    дата отправки, 
*    тема письма, 
*    текст письма полный  
)

Логин тестового ящика: **study.ai_172@mail.ru**  
Пароль тестового ящика: **NextPassword172#**

# Вариант II

Написать программу, которая собирает товары «В тренде» с сайта техники mvideo и складывает данные в БД. Сайт можно выбрать и свой. Главный критерий выбора: динамически загружаемые товары


In [1]:
import environs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common import exceptions
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date
import time
from pprint import pprint

In [2]:
env = environs.Env()
env.read_env('.env')
LOGIN = env('LOGIN')
PASSWORD = env('PASSWORD')

In [3]:
options = Options()
options.add_argument('--start-maximised')

In [4]:
driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
driver.implicitly_wait(10)
driver.fullscreen_window()
driver.get('https://mail.ru/')

In [5]:
el = driver.find_element(By.CLASS_NAME, 'email-input')
el.send_keys(LOGIN)

In [6]:
el = driver.find_element(By.XPATH, '//button[@data-testid="enter-password"]')
el.click()

In [7]:
el = driver.find_element(By.CLASS_NAME, 'password-input')
el.send_keys(PASSWORD)
el.send_keys(Keys.ENTER)

In [8]:
letters = driver.find_elements(By.CLASS_NAME, 'js-tooltip-direction_letter-bottom')
letters[0].click()

In [9]:
letters_list = []
i = 0
while True:
    letter_data = {}
    lnk = driver.current_url
    if i == 1 and lnk == link:
        break
    link = driver.current_url
    thema = driver.find_element(By.CLASS_NAME, 'thread__subject').text
    mail_from = driver.find_element(By.CLASS_NAME, 'letter-contact').text
    letter_date = driver.find_element(By.CLASS_NAME, 'letter__date').text
    if 'Сегодня' in letter_date:
        letter_date = date.today().strftime('%d, %B')
    letter_text = driver.find_element(By.CLASS_NAME, 'letter__body').text
    letter_data['E-mail отправителя'] = mail_from
    letter_data['Дата получения'] = letter_date
    letter_data['Тема письма'] = thema
    letter_data['Текст письма'] = letter_text
    letter_data['Ссылка на письмо'] = link
    letters_list.append(letter_data)
    next_letter = driver.find_element(By.CLASS_NAME, 'portal-menu-element_next')
    next_letter.click()
    i = 1
    time.sleep(3)
   

In [10]:
len(letters_list)

1042

In [11]:
from pymongo import MongoClient as mcl

In [12]:
client = mcl('127.0.0.1', 27017)
db = client['letters']
letters = db.letters
letters.create_index('link', unique=True)
for doc in letters_list:
    try:
        news.insert_one(doc)
    except:
        continue
